In [1]:
import pandas as pd
data=pd.read_csv("purchases.txt",header = None, sep = '\t')
data.columns=['customer_id', 'purchase_amount', 'date_of_purchase']

In [2]:
data.head()

,customer_id,purchase_amount,date_of_purchase
0,760,25.0,2009-11-06
1,860,50.0,2012-09-28
2,1200,100.0,2005-10-25
3,1420,50.0,2009-07-09
4,1940,70.0,2013-01-25


In [3]:
from datetime import datetime
data["date_of_purchase"]=data["date_of_purchase"].apply(lambda x:datetime.strptime(x, "%Y-%m-%d"))

In [4]:
snapshot_date=datetime.strptime("2016-01-01", "%Y-%m-%d")
data["days_since"]=data["date_of_purchase"].apply(lambda x:(snapshot_date-x).days)
data.head()
#data[data.columns[1:]].describe()


,customer_id,purchase_amount,date_of_purchase,days_since
0,760,25.0,2009-11-06,2247
1,860,50.0,2012-09-28,1190
2,1200,100.0,2005-10-25,3720
3,1420,50.0,2009-07-09,2367
4,1940,70.0,2013-01-25,1071


In [ ]:
import numpy as np
datamart = data.groupby(['customer_id']).agg({
'days_since': 'min','customer_id': 'count',
'purchase_amount': 'mean'})
datamart.columns=["recency",'frequency','amount']
datamart["amount"]=datamart["amount"].apply(lambda x:round(x,2))
datamart["frequency"]=datamart["frequency"].apply(lambda x:np.log(x))
from sklearn.preprocessing import StandardScaler 
scaler = StandardScaler() 
scaler.fit(datamart) 
datamart_normalized = scaler.transform(datamart)
datamart_normalized = pd.DataFrame(data=datamart_normalized, index=datamart.index, columns=datamart.columns)
datamart.head()

In [12]:
len(pd.unique(data["customer_id"]))

18417

In [26]:
from sklearn.cluster import AgglomerativeClustering
clustering = AgglomerativeClustering(linkage='ward',affinity='euclidean',n_clusters=5).fit(datamart)
cluster_labels=clustering.labels_
datamart=datamart.assign(Cluster = cluster_labels)

In [27]:
datamart.head()

,recency,frequency,amount,Cluster
customer_id,,,,
10,3829,0.000000,30.00,3
80,343,1.945910,71.43,1
90,758,2.302585,115.80,1
120,1401,0.000000,20.00,2
130,2970,0.693147,50.00,3


In [28]:
datamart.groupby(['Cluster']).agg({'recency': 'mean','frequency': 'mean','amount': ['mean','count'],}).round(0)

recency frequency amount      
           mean      mean   mean count
Cluster                               
0        2166.0       0.0   55.0  3891
1         534.0       1.0   93.0  3960
2        1238.0       1.0   42.0  3439
3        3125.0       0.0   36.0  2548
4          69.0       1.0   54.0  4579